In [1]:
%load_ext autoreload
%autoreload 2

## Example how to make report

In [2]:
from reporter import Reporter
from reporter.config import DLConfig
from pathlib import Path
import json

In [3]:
pwd

'/home/jovyan/OCR_3379'

In [5]:
artifacts_dir = work_dir / 'artifacts'

#### Utils

In [6]:
def load_json(filename):
    """Считывает JSON-файл и возвращает его как dict."""
    with open(filename, "r") as f:
        json_text = f.read()
    return json.loads(json_text)

#### load artifacts

In [7]:
model = Model('birth_certificate_fieldnet')

In [8]:
training = model.training(training_id=182)

In [9]:
art_list = training.artifacts

In [10]:
art_list

{'best_model.pth',
 'classes_thresholds.json',
 'config.yml',
 'model.onnx',
 'report.json',
 'run.log',
 'test_metrics.csv'}

In [11]:
training.get_artifacts(path=str(artifacts_dir))

  0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
classes_thresholds = load_json(artifacts_dir/'classes_thresholds.json')

In [12]:
classes_thresholds

{'issuer': 0.65,
 'record_number': 0.5,
 'date_of_birth': 0.675,
 'place_of_birth': 0.625,
 'doc_series': 0.625,
 'doc_number': 0.625,
 'mother_full_name': 0.65,
 'father_full_name': 0.65,
 'born_full_name': 0.6,
 'issued_place': 0.625,
 'issued_date': 0.575}

In [13]:
report_artifacts = load_json(artifacts_dir/'report.json')

In [14]:
report_artifacts

{'run_name': 'baseline/13.12/20.42.37',
 'exp_dir': '/data/smart_vision/fieldnet/runs/birth_certificate_field_temp/baseline/13.12/20.42.37',
 'sha': 'cff93e61f5d6aee37862b3777ee6801c547c2d07',
 'host_name': 'root@fieldnet-train-20231213t234222-3966c-lc2qv',
 'seed': 54321,
 'exp_name': None,
 'model_id': 327,
 'training_id': 182,
 'best_model_score': 0.8650198876857758,
 'best_model_epoch': 449,
 'classes_thresholds': {'issuer': 0.65,
  'record_number': 0.5,
  'date_of_birth': 0.675,
  'place_of_birth': 0.625,
  'doc_series': 0.625,
  'doc_number': 0.625,
  'mother_full_name': 0.65,
  'father_full_name': 0.65,
  'born_full_name': 0.6,
  'issued_place': 0.625,
  'issued_date': 0.575}}

In [15]:
config = DLConfig.load(artifacts_dir/'config.yml')

In [16]:
config['model']

{'type': 'segmentation_models_pytorch.Unet',
 'params': {'encoder_name': 'timm-mobilenetv3_large_100',
  'activation': 'softmax2d'},
 'freeze_encoder': False}

In [17]:
config['classes']

['issuer', 'record_number', 'date_of_birth', 'place_of_birth', 'doc_series', 'doc_number', 'mother_full_name', 'father_full_name', 'born_full_name', 'issued_place', 'issued_date']

### Make content for title.md template

```
Пример шаблона для title.md
```

```
# {{ title_report }}

## Модель
- Название модели в хранилище моделей - `{{ mleco_model_name }}`
- ID обучения = {{ mleco_training_idr }}

## Контакты
- Telegram: [{{ contact_telegram }}](https://t.me/{{ contact_telegram }})
- Почта: {{ contact_email }}
```

#### Content
Создаем контент для заполнения шаблона

In [18]:
content = {}

#### dict content

In [19]:
content['title_report'] = 'Сегментация полей документа свидетельсвта о рождении'
content['mleco_model_name'] = 'birthday_certificate_fieldnet'
content['training_id'] = '182'
content['contact_telegram'] = '@Kirill_Hiddelstone'
content['contact_email'] = 'kgkozlov@domclick.ru'

In [20]:
content

{'title_report': 'Сегментация полей документа свидетельсвта о рождении',
 'mleco_model_name': 'birthday_certificate_fieldnet',
 'training_id': '182',
 'contact_telegram': '@Kirill_Hiddelstone',
 'contact_email': 'kgkozlov@domclick.ru'}

#### yaml content
```yaml
title:
    title_report: 'Сегментация полей документа свидетельсвта о рождении'
    mleco_model_name: 'birthday_certificate_fieldnet'
    training_id: '182'
    contact_telegram: '@Kirill_Hiddelstone'
    contact_email: 'kgkozlov@domclick.ru'

```

### Reporter

```
templates/
    ├──base/
    │    ├──base.md
    │    ├──title.md
    │    ├──reproduce.md
    │    └──table.md
    ├──smart_vision/
    │    ├──fieldnet.md
    │    ├──rotator.md
    │    └──ocr_general.md
    └── 
```

#### report dict

In [21]:
report_content_dict = Reporter(
    templ_path ='base/title.md', # пути внтури либы
    package_name='reporter', # название либы
    env_path = 'templates' # дирректория где лежать шаблоны
)

In [23]:
# from dict
report_content_dict.report_content(context=content, output_path=str(work_dir/'test.md'))

#### output

#### report yaml/json

шаблон для yaml выглядить иначе - пример шаблона для title.md

```
# {{ title['title_report'] }}

## Модель
- Название модели в хранилище моделей - {{ title['mleco_model_name'] }}
- ID обучения = {{ title['mleco_training_id'] }}

## Контакты
- Telegram: [{{ title['contact_telegram'] }}](https://t.me/{{ title['contact_telegram'] }})
- Почта: {{ title['contact_email'] }}
```

In [32]:
report_content_dict = Reporter(
    templ_path ='title.md', # пути внтури либы
    package_name='validation_report', # название либы
    env_path = work_dir/'templates' # дирректория где лежать шаблоны
)

In [33]:
# from yaml/json
report_content_dict.report_config_content(context=artifacts_dir/'config_test.yml', output_path='test_yaml.md')